#Rag pipelines :- data ingesion to vector db pipeline

In [1]:
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

2025-12-10 10:42:43.644671: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-10 10:42:43.746714: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/dibyajit/anaconda3/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/dibyajit/anaconda3/li

In [2]:
### read all pdf's inside the directory
def process_all_pdfs (pdf_directory):
    """Process all pdf files in a directory"""
    all_documents=[]
    pdf_dir = Path(pdf_directory)
    #find all pdf files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    print(f'found {len(pdf_files)} pdf files to process \n')

    for pdf_file in pdf_files:
        print(f"processing : {pdf_file.name}\n")
        try:
            loader = PyMuPDFLoader(str(pdf_file))
            document = loader.load()
            #add source info to metadata
            for doc in document:
                doc.metadata['source_file']=pdf_file.name
                doc.metadata['file_type']='pdf'

            all_documents.extend(document)
            print(f"loaded {len(document)} pages\n")
        except Exception as e:
            print(f"error occured {e}")
    print(f"all document are loaded : {len(all_documents)}")
    return all_documents

all_pdf_documents = process_all_pdfs('../../data/pdf_files')

found 3 pdf files to process 

processing : Document from sayan_removed (1).pdf

loaded 12 pages

processing : 2712264064.pdf

loaded 3 pages

processing : socigo report.pdf

loaded 22 pages

all document are loaded : 37


In [3]:
print(all_pdf_documents[0])

page_content='82 
Possible questions from Study Materials 
QB  
Sl. No. 
Question 
Question 
Marks 
 
Module 3 
 
1 
Define Grammar with an example  
2 
2 
 Define regular grammar with an example. 
2 
3 
 Define context-free grammar with an example. 
2 
4 
 Define Chomsky Normal Form with an example 
2 
5 
 Define Greibach Normal Form with an example. 
2 
6 
 Define ambiguous grammar with an example. 
2 
7 
 Define derivation tree with an example. 
2 
8 
 Define left most derivation with an example. 
2 
9 
 Define right most derivation with an example 
2 
10 
 Define setential form with an example. 
2 
11 
 Define unit production with an example. 
2 
12 
 Define Context Sensitive Grammar with an example. 
2 
13 
 State Church Turing thesis. 
2 
14 
 Define Universal Turing Machine. 
2 
15 
 Define Universal Language. 
2 
16 
 State Ogden’s Lemma. 
2 
17 
 State Pumping Lemma for Context Free Languages 
2 
18 
 What do you mean by Halting problem of Turing Machines? 
2 
19 
 What do you

In [ ]:
### text splitting get into chinks

def split_documents(documents, chunk_size=1000,chunk_overlap = 200):
    """split documents into smaller chunks for better rag performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function = len,
        separators = ["\n\n","\n"," ",""]
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    if split_docs:
        print(f"\n example chunks")
        print(f"Content :{split_docs[0].page_content[:200]}...")
        print(f"Metadata:{split_docs[0].metadata}")
    else:
        print("error occured in chunking")
    return split_docs

In [5]:
chunks = split_documents(all_pdf_documents)

Split 37 documents into 72 chunks

 example chunks
Content :82 
Possible questions from Study Materials 
QB  
Sl. No. 
Question 
Question 
Marks 
 
Module 3 
 
1 
Define Grammar with an example  
2 
2 
 Define regular grammar with an example. 
2 
3 
 Define co...
Metadata:{'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': '../../data/pdf_files/Document from sayan_removed (1).pdf', 'file_path': '../../data/pdf_files/Document from sayan_removed (1).pdf', 'total_pages': 12, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-10-27T15:19:09+00:00', 'trapped': '', 'modDate': 'D:20251027151909Z', 'creationDate': '', 'page': 0, 'source_file': 'Document from sayan_removed (1).pdf', 'file_type': 'pdf'}


Embedding and vectorStoreDB

In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
class EmbeddingManager:
    """Handels document embedding generation using SentenceTransformers"""
    
    def __init__(self,model_name:str ="sentence-transformers/all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model name: Hugging face model name for system embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the sentence transformer model"""
        try:
            print(f"Loading embedding model {self.model_name}\n")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully , Embedding dimensions:{self.model.get_sentence_embedding_dimension()}\n")
        except Exception as e:
            print(f"Error loading model{self.model_name} {e}\n")
            raise 

    def generate_embeddings(self,texts:List[str])-> np.ndarray:
        """Generate embeddings for  a list of strings"""

        if not self.model:
            raise ValueError("Model not  loaded")
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape {embeddings.shape}")
        return embeddings
    
    # def get_sentence_embedding_dimension(self)-> int:
    #     """Get the embedding dimensions of the model"""

    #     if not self.model:
    #         raise ValueError("Model not loaded")
    #     return self.model.get_sentence_embedding_dimension()

    #initialize the embedding manager

embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model sentence-transformers/all-MiniLM-L6-v2

Model loaded successfully , Embedding dimensions:384



Vector store

In [8]:
class VectorStore:
    """Manages document embeddings in a chromaDB vector store"""
    def __init__(self,collection_name:str = "pdf_documents",persist_directory : str = "../../data/vector_store"):
        """Initialize the vector store"""

        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self.initialize_store()
    def initialize_store(self):
        """Initialize chromaDB client and collection"""
        try:
            #create persistent chroma client
            os.makedirs(self.persist_directory,exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            #get or create collection
            self.collection = self.client.get_or_create_collection(
                name= self.collection_name,
                metadata={"description":"pdf document embeddings for rag"}
            )
            print(f"Vector store initialized, Collection:{self.collection_name}")
            print(f"Exiting documents in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error initializing vector store :{e}")
            raise

    def add_documents(self,documents:List[Any],embeddings:np.ndarray):
        """ Add documents and other embeddings to the vedtor db
            Args:
            documents: List of langchain documents
            embeddings: Corresponding embeddings to the documents
        """
        if len(documents)!=len(embeddings):
            raise ValueError("Number of documents must be equal to the number of embeddings")
        print(f"adding {len(documents)} documents to the vector store")

        #prepare data for chromaDB
        ids=[]
        metadatas =[]
        documents_text =[]
        embeddings_list=[]

        for i,(doc,embedding) in enumerate(zip(documents,embeddings)):
            #generate unique id
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            #Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index']=i
            metadata['content_length']=len(doc.page_content)
            metadatas.append(metadata)

            #document content
            documents_text.append(doc.page_content)

            #Embedding
            embeddings_list.append(embedding.tolist())
        
        #add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"successfully added {len(documents)} documents to vector store")
            print(f"Total document in collection {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store :{e}")
            raise

vectorstore = VectorStore();
vectorstore


Vector store initialized, Collection:pdf_documents
Exiting documents in collection: 400


In [9]:
#convert the chunks to embeddings
texts = [doc.page_content for doc in chunks]
#Generate the embeddings
embeddings = embedding_manager.generate_embeddings(texts)
#store in the vector database
vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 72 texts...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Generated embeddings with shape (72, 384)
adding 72 documents to the vector store
successfully added 72 documents to vector store
Total document in collection 472


Retrever pipeline for VectorStore

In [10]:
class RAGRetriver:
    def __init__(self,vector_store:VectorStore,embedding_manager:EmbeddingManager):
        """ Initialize the retriver
        args:
            vector store: vector store contains document embeddings
            embedding manager: manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self,query:str,top_k:int=5,score_threshold:float=0.0)->List[Dict[str,Any]]:
        """ Retrive relavent documents for a query
        Args:
            query: The search query
            top_k: Number of top results to return 
            score_threshold: Minimum similarity score threshold
        Returns:
            List of dictionaries containing retrived documents and metadata
        """
        print(f"Retriving documents for query{query}")
        print(f"Top k: {top_k},Score threshold:{score_threshold}")

        #generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        #search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings = [query_embedding.tolist()],
                n_results = top_k
            )
            #process results
            retrived_docs =[]

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i , (document, metadata, distance, id) in enumerate(zip(documents,metadatas,distances,ids)):
                    #convert distance to similarity score
                    similarity_score = 1 - distance

                    if similarity_score>=score_threshold:
                        retrived_docs.append({
                            'id':id,
                            'content':document,
                            'metadata':metadata,
                            'similarity_score':similarity_score,
                            'distance':distance,
                            'rank':i+1
                        })
                print(f"Retrived {len(retrived_docs)} documents (after filtering)")
            else:
                print("No documents found")
            return retrived_docs
        except Exception as e:
            print(f"Error during retrival {e}")
            return []
        
rag_retriver = RAGRetriver(vectorstore,embedding_manager)

In [11]:
rag_retriver.retrieve("This is to certify that the project report entitled “SociGo-Social Media Platform” submitted by")

Retriving documents for queryThis is to certify that the project report entitled “SociGo-Social Media Platform” submitted by
Top k: 5,Score threshold:0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape (1, 384)
Retrived 5 documents (after filtering)


[{'id': 'doc_cae6eac7_59',
  'content': 'III \n \nApproval Certificate \n \n \nThis is to certify that the project report entitled “SociGo-Social Media Platform” submitted by \nArpan Pramanik(Roll:12023002001104), Dibyajit Nandi(Roll:12023002001072) and Sanglap \nGhosh (Roll:12023002001057)  in partial fulfillment of the requirements of the degree of \nBachelor of Technology in Computer Science & Engineering from University of Engineering \nand Management, Jaipur was conducted in a systematic and procedural manner to the best of our',
  'metadata': {'format': 'PDF 1.7',
   'content_length': 488,
   'creationdate': '2025-11-06T22:54:21+05:30',
   'producer': '3.0.30 (5.1.14)',
   'author': 'WPS Office',
   'keywords': '',
   'title': '',
   'creationDate': "D:20251106225421+05'30'",
   'source_file': 'socigo report.pdf',
   'page': 2,
   'trapped': '',
   'doc_index': 59,
   'source': '../../data/pdf_files/socigo report.pdf',
   'total_pages': 22,
   'creator': 'Microsoft® Word 2021',
 

Integration vector db context pipeline output with llm output

In [47]:
#simple rag pipeline with groq llm
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

#Initialize the groq LLM
groq_api_key = os.getenv("GROQ_API")
llm = ChatGroq(api_key=groq_api_key,model ="openai/gpt-oss-120b",temperature=0.1,max_tokens=1024)

#Simple rag function to retrive context to generate response
def rag_simple(query, retriever, llm,top_k=3):
    ##retrive the context
    results = retriever.retrieve(query,top_k=top_k)
    context = "\n\n".join(doc['content'] for doc in results) if results else ""
    if not context:
        print("No relavent context found\n")
    #Generate the answer using groq llm
    prompt =f"""Use the following context to answer the questions concisely.
        Context:
        {context}

        Question: {query}
        Answer:"""
    
    response = llm.invoke([prompt.format(context =context, query=query)])
    return response.content

In [52]:
answer = rag_simple("This project presents SociGo-a cross-platform,",rag_retriver,llm)
print(answer)

Retriving documents for queryThis project presents SociGo-a cross-platform,
Top k: 3,Score threshold:0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape (1, 384)
Retrived 2 documents (after filtering)
This project presents SociGo—a cross‑platform social media platform built with a modular architecture to meet evolving user demands and support future AI‑driven features.
